# Init

In [1]:
import pandas as pd

In [2]:
cd ..

/Users/wliao0504/Library/Mobile Documents/com~apple~CloudDocs/Repo/hca-lab/CLIF-MIMIC


/Users/wliao0504/.pyenv/versions/3.13.0/lib/python3.13/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
pwd

'/Users/wliao0504/Library/Mobile Documents/com~apple~CloudDocs/Repo/hca-lab/CLIF-MIMIC'

In [4]:
from src.utils import check_duplicates, find_duplicates, read_from_rclif, clif_table_pathfinder, \
    mimic_table_pathfinder

from importlib import reload
import src.tables.labs
reload(src.tables.labs)
from src.tables.labs import LabsPipeline

loaded configuration from /Users/wliao0504/Library/Mobile Documents/com~apple~CloudDocs/Repo/hca-lab/CLIF-MIMIC/src/../config/config.json


# Dev

## new pipeline

In [5]:
pipeline = LabsPipeline()
pipeline.run()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

ERROR:etl.labs:Error in labs pipeline: IO Error: Could not read from file "/Users/wliao0504/Library/Mobile Documents/com~apple~CloudDocs/Repo/hca-lab/CLIF-MIMIC/data/mimic-data/mimic-iv-3.1/parquet-dev-test/chartevents.parquet": Operation timed out


IOException: IO Error: Could not read from file "/Users/wliao0504/Library/Mobile Documents/com~apple~CloudDocs/Repo/hca-lab/CLIF-MIMIC/data/mimic-data/mimic-iv-3.1/parquet-dev-test/chartevents.parquet": Operation timed out

In [11]:
pipeline.data.keys()

dict_keys(['df_le', 'df_ce', 'labs_items', 'df_f'])

In [24]:
df_f = pipeline.data['df_f']

## dedup

In [14]:
labs_events_f = read_from_rclif("labs_intm")

In [15]:
len(labs_events_f)

46174325

In [4]:
labs_dups = find_duplicates(labs_events_f, ["hospitalization_id", "lab_collect_dttm", "lab_result_dttm", "lab_category"]) \
    .sort_values(["hospitalization_id", "lab_collect_dttm", "lab_result_dttm", "lab_category"])

labs_dups

,hospitalization_id,lab_order_dttm,lab_collect_dttm,lab_result_dttm,lab_order_name,lab_order_category,lab_name,lab_category,lab_value,lab_value_numeric,reference_unit,lab_specimen_name,lab_specimen_category,lab_loinc_code
9683715,20000147,NaT,2121-08-30 17:38:00,2121-08-30 17:39:00,,,pCO2,pco2_arterial,35,35.00,mm Hg,,,
44761220,20000147,NaT,2121-08-30 17:38:00,2121-08-30 17:39:00,,,Arterial CO2 Pressure,pco2_arterial,35,35.00,mmHg,,,
9683716,20000147,NaT,2121-08-30 17:38:00,2121-08-30 17:39:00,,,pH,ph_arterial,7.41,7.41,units,,,
44762337,20000147,NaT,2121-08-30 17:38:00,2121-08-30 17:39:00,,,PH (Arterial),ph_arterial,7.41,7.41,units,,,
9683717,20000147,NaT,2121-08-30 17:38:00,2121-08-30 17:39:00,,,pO2,po2_arterial,390,390.00,mm Hg,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44775215,29999625,NaT,2157-11-24 02:37:00,2157-11-24 02:40:00,,,Arterial CO2 Pressure,pco2_arterial,43,43.00,mmHg,,,
4567471,29999625,NaT,2157-11-24 02:37:00,2157-11-24 02:40:00,,,pH,ph_arterial,7.41,7.41,units,,,
44776501,29999625,NaT,2157-11-24 02:37:00,2157-11-24 02:40:00,,,PH (Arterial),ph_arterial,7.41,7.41,units,,,
4567472,29999625,NaT,2157-11-24 02:37:00,2157-11-24 02:40:00,,,pO2,po2_arterial,81,81.00,mm Hg,,,


In [5]:
# Drop trivial duplicates
labs_events_fd = labs_events_f.drop_duplicates(
        subset = ["hospitalization_id", "lab_collect_dttm", "lab_result_dttm", "lab_category", "lab_value_numeric"],
        inplace = False)

After we drop the trivial duplicates (where the values are the same and thus simply repeated), we are left with 4015 non-trivial cases: 

In [6]:
labs_dups_d = find_duplicates(labs_events_fd, ["hospitalization_id", "lab_collect_dttm", "lab_result_dttm", "lab_category"]) \
    .sort_values(["hospitalization_id", "lab_collect_dttm", "lab_result_dttm", "lab_category"])

labs_dups_d

,hospitalization_id,lab_order_dttm,lab_collect_dttm,lab_result_dttm,lab_order_name,lab_order_category,lab_name,lab_category,lab_value,lab_value_numeric,reference_unit,lab_specimen_name,lab_specimen_category,lab_loinc_code
18566630,20012928,NaT,2123-03-02 04:45:00,2123-03-02 05:48:00,,,Hemoglobin,hemoglobin,7.7,7.7,g/dL,,,
18566642,20012928,NaT,2123-03-02 04:45:00,2123-03-02 05:48:00,,,Hemoglobin,hemoglobin,7.6,7.6,g/dL,,,
18566631,20012928,NaT,2123-03-02 04:45:00,2123-03-02 05:48:00,,,Platelet Count,platelet_count,148,148.0,K/uL,,,
18566643,20012928,NaT,2123-03-02 04:45:00,2123-03-02 05:48:00,,,Platelet Count,platelet_count,139,139.0,K/uL,,,
38581298,20025172,NaT,2139-12-01 08:38:00,2139-12-01 10:31:00,,,Hemoglobin,hemoglobin,7.7,7.7,g/dL,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22382782,29983564,NaT,2128-06-07 06:47:00,2128-06-07 08:42:00,,,PT,pt,12.3,12.3,sec,,,
22382766,29983564,NaT,2128-06-07 06:47:00,2128-06-07 08:42:00,,,PTT,ptt,28.4,28.4,sec,,,
22382783,29983564,NaT,2128-06-07 06:47:00,2128-06-07 08:42:00,,,PTT,ptt,28.7,28.7,sec,,,
44677336,29984509,NaT,2158-10-22 12:00:00,2158-10-22 12:04:00,,,Mixed Venous O2% Sat,so2_mixed_venous,42,42.0,%,,,


If we discard duplicates that result from many to one mapping (which we can solve by assigning duduplication rank), we are left with the case of geneinue duplicates from the same item (displayed below). Luckily, most of them seem to be close-value duplicates (which we may define as being within 10% from each other). Then the remaining are the most thorny cases: observations of the same item of the same patient at same time that differ dramatically in values. Should we drop either? both? Or are they legit duplicates from two different samples / specimens?
 

In [7]:
labs_dups_d2 = find_duplicates(labs_events_fd, ["hospitalization_id", "lab_collect_dttm", "lab_result_dttm", "lab_name"]) \
    .sort_values(["hospitalization_id", "lab_collect_dttm", "lab_result_dttm", "lab_category"])

In [8]:
# remove some empty columns for better display
labs_dups_d2.drop(
    columns = ["lab_order_dttm", "lab_order_name", "lab_order_category", "lab_specimen_name", "lab_specimen_category", "lab_loinc_code"],
    inplace = True
    )
labs_dups_d2

,hospitalization_id,lab_collect_dttm,lab_result_dttm,lab_name,lab_category,lab_value,lab_value_numeric,reference_unit
18566630,20012928,2123-03-02 04:45:00,2123-03-02 05:48:00,Hemoglobin,hemoglobin,7.7,7.7,g/dL
18566642,20012928,2123-03-02 04:45:00,2123-03-02 05:48:00,Hemoglobin,hemoglobin,7.6,7.6,g/dL
18566631,20012928,2123-03-02 04:45:00,2123-03-02 05:48:00,Platelet Count,platelet_count,148,148.0,K/uL
18566643,20012928,2123-03-02 04:45:00,2123-03-02 05:48:00,Platelet Count,platelet_count,139,139.0,K/uL
38581298,20025172,2139-12-01 08:38:00,2139-12-01 10:31:00,Hemoglobin,hemoglobin,7.7,7.7,g/dL
...,...,...,...,...,...,...,...,...
39224708,29977045,2134-12-23 11:35:00,2134-12-23 13:16:00,Hemoglobin,hemoglobin,6.9,6.9,g/dL
22382765,29983564,2128-06-07 06:47:00,2128-06-07 08:42:00,PT,pt,12.4,12.4,sec
22382782,29983564,2128-06-07 06:47:00,2128-06-07 08:42:00,PT,pt,12.3,12.3,sec
22382766,29983564,2128-06-07 06:47:00,2128-06-07 08:42:00,PTT,ptt,28.4,28.4,sec


In [9]:
labs_dups_d2

,hospitalization_id,lab_collect_dttm,lab_result_dttm,lab_name,lab_category,lab_value,lab_value_numeric,reference_unit
18566630,20012928,2123-03-02 04:45:00,2123-03-02 05:48:00,Hemoglobin,hemoglobin,7.7,7.7,g/dL
18566642,20012928,2123-03-02 04:45:00,2123-03-02 05:48:00,Hemoglobin,hemoglobin,7.6,7.6,g/dL
18566631,20012928,2123-03-02 04:45:00,2123-03-02 05:48:00,Platelet Count,platelet_count,148,148.0,K/uL
18566643,20012928,2123-03-02 04:45:00,2123-03-02 05:48:00,Platelet Count,platelet_count,139,139.0,K/uL
38581298,20025172,2139-12-01 08:38:00,2139-12-01 10:31:00,Hemoglobin,hemoglobin,7.7,7.7,g/dL
...,...,...,...,...,...,...,...,...
39224708,29977045,2134-12-23 11:35:00,2134-12-23 13:16:00,Hemoglobin,hemoglobin,6.9,6.9,g/dL
22382765,29983564,2128-06-07 06:47:00,2128-06-07 08:42:00,PT,pt,12.4,12.4,sec
22382782,29983564,2128-06-07 06:47:00,2128-06-07 08:42:00,PT,pt,12.3,12.3,sec
22382766,29983564,2128-06-07 06:47:00,2128-06-07 08:42:00,PTT,ptt,28.4,28.4,sec


In [10]:
len(labs_events_f)

46174325

In [11]:
f"{(3615 / len(labs_events_f)) * 100:.2f}%"

7.829026195835023e-05

## handle NA lab values
sometimes the `value` or `valuenum` are NA, or in a weird blank form "__". need to check if they are always accompanied by otherwise useful comments 

In [37]:
df_le = pipeline.data['df_le']
df_le.head()

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments,lab_name,lab_category
0,248852,10014610,22077651,58477112,50983,None,2171-10-02 06:10:00,2171-10-02 07:40:00,134,134.0,mEq/L,133.0,145.0,None,ROUTINE,None,Sodium,sodium
1,248853,10014610,22077651,58477112,51006,None,2171-10-02 06:10:00,2171-10-02 07:40:00,20,20.0,mg/dL,6.0,20.0,None,ROUTINE,None,Urea Nitrogen,bun
2,248876,10014610,22077651,5414949,51146,None,2171-10-02 16:00:00,2171-10-02 16:19:00,0.1,0.1,%,0.0,2.0,None,STAT,None,Basophils,basophils_percent
3,248877,10014610,22077651,5414949,51200,None,2171-10-02 16:00:00,2171-10-02 16:19:00,0.1,0.1,%,0.0,4.0,None,STAT,None,Eosinophils,eosinophils_percent
4,248879,10014610,22077651,5414949,51222,None,2171-10-02 16:00:00,2171-10-02 16:19:00,12.1,12.1,g/dL,14.0,18.0,abnormal,STAT,None,Hemoglobin,hemoglobin


In [34]:
df_le[df_le["valuenum"].isna()].value_counts('lab_name')

lab_name
Troponin T                         107398
Bilirubin, Total                    14913
PTT                                 13217
Platelet Count                      10364
INR(PT)                             10103
PT                                   9873
Alanine Aminotransferase (ALT)       9445
Hemoglobin                           5463
Basophils                            4286
Neutrophils                          4286
Eosinophils                          4283
Monocytes                            4279
Lymphocytes                          4279
Absolute Basophil Count              4002
Bilirubin, Direct                    3417
C-Reactive Protein                   2843
Urea Nitrogen                        1809
Potassium                            1428
Creatinine                           1217
Glucose                               765
pCO2                                  635
Bicarbonate                           617
Asparate Aminotransferase (AST)       568
Ferritin                 

In [36]:
valuenum_na = df_le[df_le["valuenum"].isna()]
valuenum_na.head()

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments,lab_name,lab_category
898,251817,10014651,20051301,94978785,51003,None,2138-04-27 07:58:00,2138-04-27 10:50:00,None,NaN,ng/mL,0.0,0.01,None,ROUTINE,LESS THAN 0.01. CTROPNT > 0.10 NG/ML SUGGESTS...,Troponin T,troponin_t
1319,124360,10007795,28477357,64887440,51003,None,2136-04-22 22:02:00,2136-04-23 00:52:00,None,NaN,ng/mL,0.0,0.01,None,ROUTINE,<0.01. CTROPNT > 0.10 NG/ML SUGGESTS ACUTE MI.,Troponin T,troponin_t
1902,125660,10007818,22987108,83632285,51003,None,2146-06-22 14:05:00,2146-06-22 14:52:00,None,NaN,ng/mL,0.0,0.01,None,STAT,<0.01. CTROPNT > 0.10 NG/ML SUGGESTS ACUTE MI.,Troponin T,troponin_t
2109,253061,10014652,25833444,29108039,51003,None,2155-07-31 02:45:00,2155-07-31 04:48:00,None,NaN,ng/mL,0.0,0.01,None,STAT,<0.01. cTropnT > 0.10 ng/mL suggests Acute MI.,Troponin T,troponin_t
2815,729,10000084,23052089,45668018,51003,None,2160-11-20 22:30:00,2160-11-21 00:09:00,None,NaN,ng/mL,0.0,0.01,None,STAT,<0.01. cTropnT > 0.10 ng/mL suggests Acute MI.,Troponin T,troponin_t


In [ ]:
value_na = df_le[df_le["value"].isna()]
value_na.head()

# Validate

### NA lab values

In [18]:
# clif_labs = read_from_rclif("labs")
clif_labs = pd.read_parquet(clif_table_pathfinder("labs"))

In [20]:
clif_labs[clif_labs["lab_value"].isna()].value_counts('lab_name')

lab_name
Troponin T                         107388
Bilirubin, Total                    14912
PTT                                 12542
Platelet Count                      10364
INR(PT)                              9799
PT                                   9602
Alanine Aminotransferase (ALT)       9444
Hemoglobin                           5463
Neutrophils                          4286
Monocytes                            4279
Lymphocytes                          4279
Basophils                            4279
Eosinophils                          4279
Absolute Basophil Count              4002
Bilirubin, Direct                    3417
C-Reactive Protein                   2843
Urea Nitrogen                        1808
Potassium                            1428
Creatinine                           1216
Glucose                               765
pCO2                                  635
Bicarbonate                           617
Asparate Aminotransferase (AST)       567
Ferritin                 

In [23]:
# compare to the new output
df_f[df_f["lab_value"].isna()].value_counts('lab_name')

lab_name
Troponin T                         107388
Bilirubin, Total                    14912
PTT                                 12542
Platelet Count                      10364
INR(PT)                              9799
PT                                   9602
Alanine Aminotransferase (ALT)       9444
Hemoglobin                           5463
Neutrophils                          4286
Monocytes                            4279
Lymphocytes                          4279
Basophils                            4279
Eosinophils                          4279
Absolute Basophil Count              4002
Bilirubin, Direct                    3417
C-Reactive Protein                   2843
Urea Nitrogen                        1808
Potassium                            1428
Creatinine                           1216
Glucose                               765
pCO2                                  635
Bicarbonate                           617
Asparate Aminotransferase (AST)       567
Ferritin                 